# `Automated script to download Sentinel-1 images using Alaska Satellite Facility API`


# Import libraries and packages

In [1]:
!pip install asf_search
!pip install shapely
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 8.0 MB/s eta 0:00:00


In [15]:
import asf_search as asf
from shapely.geometry import box
from datetime import datetime, timezone
import os
import zipfile
import rasterio
from rasterio import shutil as rio_shutil
from google.colab import files

# Authenticate with Earthdata NASA

In [3]:
session = asf.ASFSession().auth_with_creds("carlosmendez1997", "CamvLDS1997!")

# Define GeoJSON File

In [4]:
# --- Define bounding box ---
xmin, ymin, xmax, ymax = -68.803909, -11.033737, -68.725933, -10.979773
bbox_geom = box(xmin, ymin, xmax, ymax)
bbox_wkt = bbox_geom.wkt

# Define search parameters

In [5]:
# --- Define search parameters ---
platform = "Sentinel-1A"
polarizations = ["VV", "VH"]
beam_mode = "IW"
processing_level = "GRD"
orbit_direction = "DESCENDING"
relative_orbit = 127

# Set date range

In [6]:
# --- Date range ---
start_date = "2026-01-01"
end_date = datetime.now(timezone.utc).strftime("%Y-%m-%d")

# Perform Search of S1

In [7]:
# --- Perform search ---
results = asf.search(
    platform=platform,
    processingLevel=processing_level,
    beamMode=beam_mode,
    polarization=polarizations,
    relativeOrbit=relative_orbit,
    start=start_date,
    end=end_date,
    intersectsWith=bbox_wkt
)

print(f"Found {len(results)} Sentinel-1 scenes")

Found 16 Sentinel-1 scenes


# Create folder to dowload data

In [8]:
# --- Create folder for downloads ---
os.makedirs("ASF_Sentinel1", exist_ok=True)

# Download results

In [9]:
results.download(path="ASF_Sentinel1", session=session)

/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/OPERA_L2_RTC-S1_T127-271782-IW1_20260207T101341Z_20260207T222557Z_S1C_30_v1.0.h5
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/OPERA_L2_RTC-S1_T127-271782-IW1_20260126T101341Z_20260126T191435Z_S1C_30_v1.0.h5
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/OPERA_L2_RTC-S1_T127-271782-IW1_20260108T101442Z_20260108T164159Z_S1A_30_v1.0.h5
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/down

# Create folder only with Geotiff files

In [10]:
# --- Carpeta de salida para GeoTIFF ---
output_dir = "ASF_Sentinel1/GeoTIFFs"
os.makedirs(output_dir, exist_ok=True)

# --- Filtrar y copiar solo los .tiff (VV/VH) ---
for root, dirs, files in os.walk("ASF_Sentinel1"):
    for f in files:
        if f.endswith(".tiff") and ("VV" in f or "VH" in f):
            src_path = os.path.join(root, f)
            try:
                date_str = f.split("_")[3][:8]  # Extraer fecha (YYYYMMDD)
            except Exception:
                date_str = "unknown"
            pol = "VV" if "VV" in f else "VH"
            out_name = f"S1_{pol}_{date_str}.tif"
            out_path = os.path.join(output_dir, out_name)

            # Copiar GeoTIFF a carpeta de salida
            rio_shutil.copy(src_path, out_path, driver='GTiff')
            print(f"Guardado {out_path}")

Guardado ASF_Sentinel1/GeoTIFFs/S1_VV_20260126.tif
Guardado ASF_Sentinel1/GeoTIFFs/S1_VH_20260108.tif
Guardado ASF_Sentinel1/GeoTIFFs/S1_VV_20260102.tif
Guardado ASF_Sentinel1/GeoTIFFs/S1_VH_20260207.tif
Guardado ASF_Sentinel1/GeoTIFFs/S1_VH_20260126.tif
Guardado ASF_Sentinel1/GeoTIFFs/S1_VH_20260102.tif
Guardado ASF_Sentinel1/GeoTIFFs/S1_VV_20260108.tif
Guardado ASF_Sentinel1/GeoTIFFs/S1_VV_20260207.tif


In [11]:
# --- Comprimir todos los GeoTIFF en un único ZIP ---
zip_path = "ASF_Sentinel1/Sentinel1_GeoTIFFs.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        zipf.write(file_path, arcname=file)

print(f"✅ Archivo comprimido listo: {zip_path}")

✅ Archivo comprimido listo: ASF_Sentinel1/Sentinel1_GeoTIFFs.zip


In [13]:
# --- Comprimir todos los GeoTIFF en un único ZIP ---
zip_path = "ASF_Sentinel1/Sentinel1_GeoTIFFs.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        zipf.write(file_path, arcname=file)

print(f"✅ Archivo comprimido listo: {zip_path}")

✅ Archivo comprimido listo: ASF_Sentinel1/Sentinel1_GeoTIFFs.zip


In [16]:
# --- Descargar ZIP a máquina local ---
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>